In [20]:
from pandas.io import sql
import os
import pandas as pd
import matplotlib as mpl
import re
import datetime
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import font_manager, rc
import sys
pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', None) ## 모든 열 출력
# 출력 옵션 설정
pd.set_option('display.max_colwidth', -1)

/var/folders/0m/g1f73mzn5qd3rgpzy2nyg0zc0000gn/T/ipykernel_42419/22682425.py:15: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


1. Laser Layer 완료
2. LASER : Bulid, Blade, Left, Blade, Melting Layer(Poly), Melting Layer(Hatch)
3. TEST : Melting Layer

In [27]:
# 1. Laser Layer 완료 처리 코드
# 파일들의 경로 리스트
log_path = f'./20231014'
log_file_paths = [
    
    '09.log',
    '10.log'
]
combined_dfs = []

# 모든 로그 파일을 순회
for log_file_path in log_file_paths:
    log_lines_list = []

    # 로그 파일을 읽어 로그 라인들을 저장
    with open(f'{log_path}/{log_file_path}', 'r', encoding='euc-kr') as log_file:
        log_lines = [line.strip() for line in log_file]
        log_lines_list.extend(log_lines)

    # 로그 라인들을 데이터프레임에 저장
    log_df = pd.DataFrame({'log_line': log_lines_list})

    log_df_cli = log_df["log_line"].str.split(',', expand=True)

    # Layer별 최대, 최소 시간 추출
    log_df_cli["timestamp"] = log_df_cli[0].str.slice(3, 27)
    log_df_cli["CaseID"] = log_df_cli[1].str.replace('Laser Layer 완료','')
    log_df_cli["Activity"] = log_df_cli[1].str.slice(-14,)

    #----------------------------------------------------------------------------------
    log_df_cli['Date'] = '20' + log_df_cli['timestamp'].str[1:10]
    #----------------------------------------------------------------------------------
    
    log_df_cli['Timestamp'] = log_df_cli['Date'] + ' ' + log_df_cli['timestamp'].str[10:18] + '.'+ log_df_cli['timestamp'].str[20:]
    log_df_cli['Timestamp'] = log_df_cli['Timestamp'].str.replace('  ',' ')
    # Timestamp 컬럼을 datetime 형식으로 변환
    log_df_cli['Timestamp'] = pd.to_datetime(log_df_cli['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
    
    log_df_cli_result = log_df_cli[['CaseID', 'Activity','Timestamp']]
    
    combined_dfs.append(log_df_cli_result)
    
# 모든 처리 결과를 하나의 데이터프레임으로 합침
combined_result_df_add = pd.concat(combined_dfs, ignore_index=True)

combined_result_df_add = combined_result_df_add[combined_result_df_add['Activity'] == 'Laser Layer 완료'].reset_index(drop=True)
final_df = combined_result_df_add
final_df['Timestamp'] = pd.to_datetime(final_df['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

final_df['CaseID'] = final_df['CaseID'].astype(int)
final_df = final_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
final_df = final_df.sort_values(by=['CaseID','Timestamp']).reset_index(drop=True)

df_1 = final_df

In [28]:
log_df

,log_line
0,"5 : 23-10-14 10:00:14:0606 LAYER, 119Laser Layer 완료"
1,"5 : 23-10-14 10:00:31:0463 LAYER, 120Laser Layer 완료"
2,"5 : 23-10-14 10:00:48:0318 LAYER, 121Laser Layer 완료"
3,"5 : 23-10-14 10:01:05:0178 LAYER, 122Laser Layer 완료"
4,"5 : 23-10-14 10:01:22:0025 LAYER, 123Laser Layer 완료"
...,...
209,"5 : 23-10-14 10:58:14:0751 LAYER, 328Laser Layer 완료"
210,"5 : 23-10-14 10:58:31:0191 LAYER, 329Laser Layer 완료"
211,"5 : 23-10-14 10:58:47:0642 LAYER, 330Laser Layer 완료"
212,"5 : 23-10-14 10:59:04:0099 LAYER, 331Laser Layer 완료"


In [29]:
df_1

,CaseID,Activity,Timestamp
0,3,Laser Layer 완료,2023-10-14 09:28:48.674
1,4,Laser Layer 완료,2023-10-14 09:29:04.800
2,5,Laser Layer 완료,2023-10-14 09:29:20.920
3,6,Laser Layer 완료,2023-10-14 09:29:37.049
4,7,Laser Layer 완료,2023-10-14 09:29:53.174
...,...,...,...
325,328,Laser Layer 완료,2023-10-14 10:58:14.751
326,329,Laser Layer 완료,2023-10-14 10:58:31.191
327,330,Laser Layer 완료,2023-10-14 10:58:47.642
328,331,Laser Layer 완료,2023-10-14 10:59:04.099


In [30]:
# 2. Laser 처리 코드
# 파일들의 경로 리스트
log_path = f'./20231014/LASER'
log_file_paths = [
    
    '09.log',
    '10.log'
]
combined_dfs_1 = []
combined_dfs_2 = []

# 모든 로그 파일을 순회
for log_file_path in log_file_paths:
    log_lines_list = []

    # 로그 파일을 읽어 로그 라인들을 저장
    with open(f'{log_path}/{log_file_path}', 'r', encoding='euc-kr') as log_file:
        log_lines = [line.strip() for line in log_file]
        log_lines_list.extend(log_lines)

    # 로그 라인들을 데이터프레임에 저장
    log_df = pd.DataFrame({'log_line': log_lines_list})

    log_df_cli = log_df["log_line"].str.split(',', expand=True)
    
    #Bulid, Blade, Left, Blade / Melting Layer(Poly), Melting Layer(Hatch) 분할
    log_df_cli_1 = log_df_cli[log_df_cli[7].isnull()].reset_index(drop=True)
    log_df_cli_2 = log_df_cli[~log_df_cli[7].isnull()].reset_index(drop=True)
    
    #----------------------------------------------------------------------------------
    log_df_cli_1["Activity"] = log_df_cli_1[0].str.slice(28, -4)
    log_df_cli_1["timestamp"] = log_df_cli_1[0].str.slice(14, 27)
    log_df_cli_1['timestamp'] = log_df_cli_1['timestamp'].str[:8] + ':' +log_df_cli_1['timestamp'].str[10:]
    log_df_cli_1 = log_df_cli_1.rename(columns={0:'Del1',1:'Axis_X',2:'Axis_Y'})
    log_df_cli_1['Axis_X'] = log_df_cli_1['Axis_X'].str.replace('X : ','')
    log_df_cli_1['Axis_Y'] = log_df_cli_1['Axis_Y'].str.replace('Y : ','')
  
    log_df_cli_1['Date'] = '20' + log_df_cli_1['Del1'].str.slice(5,13)
    log_df_cli_1['CaseID'] = log_df_cli_1['Del1'].str.split(' : ', expand=True)[2]
    log_df_cli_1['Timestamp'] = log_df_cli_1['Date'] + ' ' + log_df_cli_1['timestamp']
    log_df_cli_1['Timestamp'] = pd.to_datetime(log_df_cli_1['Timestamp'], format='%Y-%m-%d %H:%M:%S:%f')
    log_df_cli_1 = log_df_cli_1[['CaseID', 'Activity', 'Timestamp', 'Axis_X', 'Axis_Y']]

    #----------------------------------------------------------------------------------
    log_df_cli_2 = log_df_cli_2.rename(columns={0:'Del1',1:'CaseID',2:'Del2',3:'Activity',4:'ActXPosition',5:'Distance_명령값',6:'Velocity',7:'Accel'})
    log_df_cli_2['Date'] = log_df_cli_2['Del1'].str.split(' ', expand=True)[2]
    log_df_cli_2['timestamp'] = log_df_cli_2['Del1'].str.split(' ', expand=True)[3]
    log_df_cli_2['timestamp'] = log_df_cli_2['timestamp'].str[:8] + ':' +log_df_cli_2['timestamp'].str[10:]
    log_df_cli_2 = log_df_cli_2[['CaseID', 'Activity', 'ActXPosition', 'Distance_명령값', 'Velocity', 'Accel','Date', 'timestamp']]
    log_df_cli_2['Timestamp'] = '20' + log_df_cli_2['Date'] + ' ' + log_df_cli_2['timestamp']
    log_df_cli_2['Timestamp'] = pd.to_datetime(log_df_cli_2['Timestamp'], format='%Y-%m-%d %H:%M:%S:%f')
    
    combined_dfs_1.append(log_df_cli_1)
    combined_dfs_2.append(log_df_cli_2)
    
combined_result_df_1 = pd.concat(combined_dfs_1, ignore_index=True)
combined_result_df_2 = pd.concat(combined_dfs_2, ignore_index=True)

combined_result_df_1 = combined_result_df_1.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df = combined_result_df_1
# Axis_X와 Axis_Y를 정수로 변환
df['Axis_X'] = df['Axis_X'].astype(float)
df['Axis_Y'] = df['Axis_Y'].astype(float)

# '사분면' 컬럼 추가 및 사분면 계산
df['Quadrant'] = np.where((df['Axis_X'] > 0) & (df['Axis_Y'] > 0), 1,
                       np.where((df['Axis_X'] > 0) & (df['Axis_Y'] < 0), 4,
                                np.where((df['Axis_X'] < 0) & (df['Axis_Y'] < 0), 3, 2)))

df['Activity_Q'] = df['Activity'] + '_' + df['Quadrant'].astype(str)
df['Activity'] = df['Activity'].str.replace(' :','')
df['Activity_Q'] = df['Activity_Q'].str.replace(' :','')

#-----------------------------------------------------------------------------------------------------------------
combined_result_df = combined_result_df_2

combined_result_df['ActXPosition'] = combined_result_df['ActXPosition'].str.replace('ActXPosition = ','')
combined_result_df['Distance_명령값'] = combined_result_df['Distance_명령값'].str.replace('Distance 명령값 = ','')
combined_result_df['Velocity'] = combined_result_df['Velocity'].str.replace('Velocity = ','')
combined_result_df['Accel'] = combined_result_df['Accel'].str.replace('Accel = ','')
combined_result_df['Activity'] = combined_result_df['Activity'].str[:-2]
combined_result_df = combined_result_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
combined_result_df['Activity'] = combined_result_df['Activity'].str.replace(' :','')
combined_result_df_blade_Y = combined_result_df[combined_result_df['Activity'] == 'Blade']
combined_result_df_blade_N = combined_result_df[combined_result_df['Activity'] != 'Blade']

# Num 컬럼 추가 및 초기값 설정
combined_result_df_blade_Y['Num'] = 1

# CaseID로 그루핑한 후 'Num' 열에 누적값 계산
combined_result_df_blade_Y['Num'] = combined_result_df_blade_Y.groupby('CaseID')['Num'].cumsum()

combined_result_df_blade_Y['Activity'] = combined_result_df_blade_Y['Activity'] + '_' + combined_result_df_blade_Y['Num'].astype(str)
combined_result_df_blade_Y = combined_result_df_blade_Y[['CaseID', 'Activity', 'ActXPosition', 'Distance_명령값', 'Velocity',
       'Accel', 'Timestamp']]

combined_result_df = pd.concat([combined_result_df_blade_Y,combined_result_df_blade_N])

combined_result_df = combined_result_df[['CaseID', 'Activity', 'ActXPosition', 'Distance_명령값', 'Velocity',
       'Accel', 'Timestamp']]
combined_result_df['CaseID'] = combined_result_df['CaseID'].astype(int)
combined_result_df = combined_result_df.sort_values(by=['CaseID','Timestamp'])

df_2 = pd.concat([df,combined_result_df])

/var/folders/0m/g1f73mzn5qd3rgpzy2nyg0zc0000gn/T/ipykernel_42419/2091489001.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_result_df_blade_Y['Num'] = 1
/var/folders/0m/g1f73mzn5qd3rgpzy2nyg0zc0000gn/T/ipykernel_42419/2091489001.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_result_df_blade_Y['Num'] = combined_result_df_blade_Y.groupby('CaseID')['Num'].cumsum()
/var/folders/0m/g1f73mzn5qd3rgpzy2nyg0zc0000gn/T/ipykernel_42419/2091489001.py:94: SettingWithCopyWarning: 
A value is 

In [31]:
log_df

,log_line
0,"10 : 23-10-14 10:00:05:0086 LAYER, 119,, Blade : 0,ActXPosition = 0, Distance 명령값 = 0, Velocity = 1000, Accel = 4000"
1,"10 : 23-10-14 10:00:05:0956 LAYER, 119,, Left : 2,ActXPosition = 0, Distance 명령값 = 100, Velocity = 800, Accel = 3000"
2,"10 : 23-10-14 10:00:13:0104 LAYER, 119,, Blade : 0,ActXPosition = 3480, Distance 명령값 = 6400, Velocity = 1000, Accel = 4000"
3,"10 : 23-10-14 10:00:13:0675 Melting Layer(Poly) : 119, X : -2.82046508789063, Y : -3.68948364257813"
4,"10 : 23-10-14 10:00:13:0735 Melting Layer(Poly) : 119, X : -2.85795593261719, Y : -3.75421142578125"
...,...
20180,"10 : 23-10-14 10:59:20:0373 Melting Layer(Hatch) : 332, X : -1.63037109375, Y : 1.92709350585938"
20181,"10 : 23-10-14 10:59:20:0373 Melting Layer(Hatch) : 332, X : -1.39356994628906, Y : 2.1507568359375"
20182,"10 : 23-10-14 10:59:20:0373 Melting Layer(Hatch) : 332, X : -1.63037109375, Y : 2.09019470214844"
20183,"10 : 23-10-14 10:59:20:0373 Melting Layer(Hatch) : 332, X : -1.53231811523438, Y : 2.23086547851563"


In [32]:
df_2

,CaseID,Activity,Timestamp,Axis_X,Axis_Y,Quadrant,Activity_Q,ActXPosition,Distance_명령값,Velocity,Accel
0,308,Melting Layer(Poly),2023-10-14 09:14:53.434,-2.99,4.64,2.00,Melting Layer(Poly)_2,NaN,NaN,NaN,NaN
1,308,Melting Layer(Poly),2023-10-14 09:14:53.434,2.09,1.71,1.00,Melting Layer(Poly)_1,NaN,NaN,NaN,NaN
2,308,Melting Layer(Poly),2023-10-14 09:14:53.434,-1.94,-3.38,3.00,Melting Layer(Poly)_3,NaN,NaN,NaN,NaN
3,308,Melting Layer(Poly),2023-10-14 09:14:53.434,3.29,0.79,1.00,Melting Layer(Poly)_1,NaN,NaN,NaN,NaN
4,308,Melting Layer(Poly),2023-10-14 09:14:53.434,0.74,2.49,1.00,Melting Layer(Poly)_1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1313,331,Blade_2,2023-10-14 10:59:02.995,NaN,NaN,NaN,NaN,9840,6400,1000,4000
1314,332,Bulid,2023-10-14 10:59:04.283,NaN,NaN,NaN,NaN,0,9870,800,3000
1315,332,Blade_1,2023-10-14 10:59:11.431,NaN,NaN,NaN,NaN,0,0,1000,4000
1316,332,Left,2023-10-14 10:59:12.301,NaN,NaN,NaN,NaN,0,100,800,3000


In [33]:
# 3. TEST : Melting Layer 처리 코드
# 파일들의 경로 리스트
log_path = f'./20231014/TEST'
log_file_paths = [
    
    '09.log',
    '10.log'
]
combined_dfs = []

# 모든 로그 파일을 순회
for log_file_path in log_file_paths:
    log_lines_list = []

    # 로그 파일을 읽어 로그 라인들을 저장
    with open(f'{log_path}/{log_file_path}', 'r', encoding='euc-kr') as log_file:
        log_lines = [line.strip() for line in log_file]
        log_lines_list.extend(log_lines)

    # 로그 라인들을 데이터프레임에 저장
    log_df = pd.DataFrame({'log_line': log_lines_list})

    log_df_cli = log_df["log_line"].str.split(',', expand=True)
    
    log_df_cli_0 = log_df_cli[0].str.split(' : ', expand=True)
    log_df_cli_1 = log_df_cli[1].str.split(' : ', expand=True)
    
    # Layer별 최대, 최소 시간 추출
    log_df_cli["timestamp"] = log_df_cli[0].str.slice(3, 27)
    log_df_cli["CaseID"] = log_df_cli_0[2]
    log_df_cli['O2_ratio'] = log_df_cli_1[1].str.replace('%','')
    log_df_cli['Activity'] = 'Melting Layer'
    
    #----------------------------------------------------------------------------------
    log_df_cli['Date'] = '20' + log_df_cli['timestamp'].str[1:10]
    #----------------------------------------------------------------------------------
    
    log_df_cli['Timestamp'] = log_df_cli['Date'] + ' ' + log_df_cli['timestamp'].str[10:18] + '.'+ log_df_cli['timestamp'].str[20:]
    log_df_cli['Timestamp'] = log_df_cli['Timestamp'].str.replace('  ',' ')
    # Timestamp 컬럼을 datetime 형식으로 변환
    log_df_cli['Timestamp'] = pd.to_datetime(log_df_cli['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
    
    log_df_cli_result = log_df_cli[['CaseID', 'Activity','Timestamp','O2_ratio']]
    
    combined_dfs.append(log_df_cli_result)
    
# 모든 처리 결과를 하나의 데이터프레임으로 합침
combined_result_df_add = pd.concat(combined_dfs, ignore_index=True)

df_3 = combined_result_df_add

In [34]:
log_df

,log_line
0,"9 : 23-10-14 10:00:13:0604 Melting Layer : 119, 산소 농도 : 0.07%"
1,"9 : 23-10-14 10:00:30:0459 Melting Layer : 120, 산소 농도 : 0.03%"
2,"9 : 23-10-14 10:00:47:0314 Melting Layer : 121, 산소 농도 : 0.07%"
3,"9 : 23-10-14 10:01:04:0174 Melting Layer : 122, 산소 농도 : 0.07%"
4,"9 : 23-10-14 10:01:21:0029 Melting Layer : 123, 산소 농도 : 0.07%"
...,...
209,"9 : 23-10-14 10:58:14:0159 Melting Layer : 328, 산소 농도 : 0.11%"
210,"9 : 23-10-14 10:58:30:0601 Melting Layer : 329, 산소 농도 : 0.11%"
211,"9 : 23-10-14 10:58:47:0044 Melting Layer : 330, 산소 농도 : 0.11%"
212,"9 : 23-10-14 10:59:03:0495 Melting Layer : 331, 산소 농도 : 0.11%"


In [35]:
df_3

,CaseID,Activity,Timestamp,O2_ratio
0,3,Melting Layer,2023-10-14 09:28:48.405,0.03
1,4,Melting Layer,2023-10-14 09:29:04.525,0.07
2,5,Melting Layer,2023-10-14 09:29:20.651,0.07
3,6,Melting Layer,2023-10-14 09:29:36.776,0.03
4,7,Melting Layer,2023-10-14 09:29:52.905,0.07
...,...,...,...,...
325,328,Melting Layer,2023-10-14 10:58:14.159,0.11
326,329,Melting Layer,2023-10-14 10:58:30.601,0.11
327,330,Melting Layer,2023-10-14 10:58:47.044,0.11
328,331,Melting Layer,2023-10-14 10:59:03.495,0.11


In [36]:
df_total = pd.concat([df_1,df_2,df_3])

df_total['CaseID'] = df_total['CaseID'].astype(int)
df_total['Timestamp'] = pd.to_datetime(df_total['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

df_total = df_total.sort_values(by=['CaseID','Timestamp']).reset_index(drop=True)

In [37]:
df_total.to_csv('result_231014.csv', index=False)

In [38]:
df_total

,CaseID,Activity,Timestamp,Axis_X,Axis_Y,Quadrant,Activity_Q,ActXPosition,Distance_명령값,Velocity,Accel,O2_ratio
0,3,Left,2023-10-14 09:28:40.755,NaN,NaN,NaN,NaN,0,100,800,3000,NaN
1,3,Blade_1,2023-10-14 09:28:47.904,NaN,NaN,NaN,NaN,0,6400,1000,4000,NaN
2,3,Melting Layer,2023-10-14 09:28:48.405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03
3,3,Melting Layer(Poly),2023-10-14 09:28:48.462,-2.99,4.64,2.00,Melting Layer(Poly)_2,NaN,NaN,NaN,NaN,NaN
4,3,Melting Layer(Poly),2023-10-14 09:28:48.462,2.09,1.71,1.00,Melting Layer(Poly)_1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
26286,332,Melting Layer(Hatch),2023-10-14 10:59:20.373,-1.39,2.15,2.00,Melting Layer(Hatch)_2,NaN,NaN,NaN,NaN,NaN
26287,332,Melting Layer(Hatch),2023-10-14 10:59:20.373,-1.63,2.09,2.00,Melting Layer(Hatch)_2,NaN,NaN,NaN,NaN,NaN
26288,332,Melting Layer(Hatch),2023-10-14 10:59:20.373,-1.53,2.23,2.00,Melting Layer(Hatch)_2,NaN,NaN,NaN,NaN,NaN
26289,332,Melting Layer(Hatch),2023-10-14 10:59:20.373,-1.63,2.25,2.00,Melting Layer(Hatch)_2,NaN,NaN,NaN,NaN,NaN
